In [1]:
from flask import redirect, url_for, session
import dash
from dash import html, dcc, dash_table
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import fileBrowserAndUploadButtonToLoadProcessStatements

import json
import plotly.express as px

# Import Flask server from keycloak-auth.py
from keycloakAuth import flaskServer, oidc

# Initialize Dash app with Flask server
app = dash.Dash(__name__, server=flaskServer, url_base_pathname='/dashboard/')

# Import SimvaBrowser from simvaWidget.py
from simvaWidget import SimvaBrowser

In [2]:
# Dash callback to handle login button click
@app.callback(
    Output('login-logout-button', 'children'),
    [Input('main', 'children')]
)
def login_logout_button(main):
    if oidc.user_loggedin:
        return "Logout"
    else:
        return "Login"
        
# Dash callback to handle login button click
@app.callback(
    Output('login-logout', 'children'),
    [Input('login-logout-button', 'n_clicks')]
)
def login_logout_button_click(n_clicks):
    if n_clicks > 0:
        return dcc.Location(pathname='/', id='login-logout-link')

# Dash callback to update connection status
@app.callback(
    Output('connection-status', 'children'),
    [Input('main', 'children')]
)
def update_connection_status(input_value):
    if oidc.user_loggedin:
        user_info = session.get('oidc_auth_profile', {})
        preferred_username = user_info.get('preferred_username')
        return f'Logged in as {preferred_username} - {session}'
    else:
        return 'Not logged in'

# Example of a protected route (not yet implemented)
@app.server.route('/dashboard/')
def render_dashboard():
    return app.index()

# Example of a logout button (not yet implemented)
@app.server.route('/logout')
def logout():
    return redirect(url_for('home'))  # Replace 'home' with your home route

In [3]:
# Dash callback to handle login button click
@app.callback(
    [Output('browser_div', 'children')],
    [Input('main', 'children')]
)
def init_storage(main):
    if oidc.user_loggedin:
        # Initialize SimvaBrowser
        storage_url = "https://minio-api.simva-beta.e-ucm.es/"  # Provide your storage URL
        global browser
        browser = SimvaBrowser(session, storage_url)
        folder_buttons = [html.Button(f, id={'type': 'folder-button', 'index': f}, n_clicks=0) for f in browser.dirs]
        file_buttons = [html.Button(f, id={'type': 'file-button', 'index': f}, n_clicks=0, style={'background-color': 'green'}) for f in browser.files if f.endswith(browser.accept)]
        run_analyse_style = {'display': 'none'}
        if not browser._isdir(browser.current_path):
            run_analyse_style = {'display': 'block'}
        appLayout = [html.Div([
            html.H3(id='current-path', children=browser.current_path),
            html.Button('..', id='parent-directory', n_clicks=0, style={'display': 'block'}),
            html.Div(id='folders-div', children=folder_buttons),
            html.Div(id='files-div', children=file_buttons),
            html.Button('Run Analyse', id='run-analyse', n_clicks=0, style=run_analyse_style)
        ])]
        return appLayout

@app.callback(
    [Output('current-path', 'children'),
     Output('folders-div', 'children'),
     Output('files-div', 'children'),
     Output('run-analyse', 'style'),
     Output('debug-browser', 'children')],
    [Input('parent-directory', 'n_clicks'),
     Input({'type': 'folder-button', 'index': dash.dependencies.ALL}, 'n_clicks'),
     Input({'type': 'file-button', 'index': dash.dependencies.ALL}, 'n_clicks')],
    [State('current-path', 'children')]
)
def update_browser(n_clicks_parent, folder_n_clicks, file_n_clicks, current_path):
    ctx = dash.callback_context
    res=f"{current_path} - triggered : {ctx.triggered} - Files : {file_n_clicks} - Folders : {folder_n_clicks} - n_clicks_parent : {n_clicks_parent}"
    print(res)
    if not ctx.triggered:
        raise PreventUpdate
    triggered_prop_id = ctx.triggered[0]['prop_id']
    # Remove the .n_clicks suffix
    cleaned_prop_id = triggered_prop_id.rsplit('.', 1)[0]
    if 'parent-directory' in cleaned_prop_id:
        if len(browser.current_path) > len(browser.base_path):
            if browser.current_path.endswith(browser.delimiter):
                browser.current_path = browser.current_path.rpartition(browser.delimiter)[0]
            browser.current_path = browser.current_path.rpartition(browser.delimiter)[0] + browser.delimiter
        else:
            browser.current_path = browser.base_path
    else:
        button_id = json.loads(cleaned_prop_id)
        if button_id['type'] == 'folder-button':
            browser.current_path = browser.current_path + button_id['index']
        elif button_id['type'] == 'file-button':
            browser.current_path = browser.current_path + button_id["index"]
    
    browser._update_files()
    
    folder_buttons = [html.Button(f, id={'type': 'folder-button', 'index': f}, n_clicks=0) for f in browser.dirs]
    file_buttons = [html.Button(f, id={'type': 'file-button', 'index': f}, n_clicks=0, style={'background-color': 'green'}) for f in browser.files if f.endswith(browser.accept)]
    run_analyse_style = {'display': 'none'} if browser._isdir(browser.current_path) else {'display': 'block'}
    
    return browser.current_path, folder_buttons, file_buttons, run_analyse_style, [html.H1(f'Res : {res}')]


@app.callback(
    Output('content', 'children'),
    [Input('run-analyse', 'n_clicks')],
    [State('current-path', 'children')]
)
def run_analyse(n_clicks, current_path):
    if n_clicks > 0:
        global players_info, xapiData, timeformats
        players_info={}
        xapiData=[]
        div_list= []
        out=[]
        err=[]
        timeformats=['%Y-%m-%dT%H:%M:%SZ','%Y-%m-%dT%H:%M:%S.%fZ']
        content_string = browser.get_file_content(current_path)
        decoded=content_string.decode('utf-8')
        try:
            fileBrowserAndUploadButtonToLoadProcessStatements.load_from_string(
                decoded, fileBrowserAndUploadButtonToLoadProcessStatements.Progress(0), players_info, xapiData, timeformats, out, err
            )
            # Normalize the JSON data to a pandas DataFrame
            import pandas as pd
            df = pd.json_normalize(xapiData)
            # Convert the DataFrame to a dictionary suitable for DataTable
            data = df.to_dict('records')
            div_list.append(html.Div([
                    html.Div(out),
                    html.Div(err),
                    html.Hr(),
                    html.Div([
                        html.H3("Length table : " + str(len(xapiData))),
                        dash_table.DataTable(
                            columns=[{"name": i, "id": i} for i in df.columns],
                            data=data
                    )])
                ]))
        except Exception as e:
            print(e)
            div_list.append(html.Div(
                html.Div([
                    'There was an error processing this file.'
                ]),
                html.Div(out),
                html.Div(err)
            ))
        
        return html.Div(div_list)
    else:
        raise PreventUpdate

In [4]:
# Layout of the dashboard
app.layout = html.Div(id="main", children=[
    html.H1('Secure Dashboard'),
    html.Button(id='login-logout-button', n_clicks=0),
    html.Div(id='login-logout'),
    html.Div(id='connection-status',children=[]),
    html.Div(id='browser_div', children=[
        html.H3(id='current-path'),
        html.Button('..', id='parent-directory', n_clicks=0, style={'display': 'none'}),
        html.Div(id='folders-div', children=[]),
        html.Div(id='files-div', children=[]),
        html.Button('Run Analyse', id='run-analyse', n_clicks=0, style={'display': 'none'})
    ]),
    html.Div(id='debug-browser', children=[]),
    html.Div(id='content',children=[])
])

In [ ]:
if __name__ == '__main__':
    app.run(debug=True, port=5000)

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:5000
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): sso.simva-beta.e-ucm.es:443
DEBUG:urllib3.connectionpool:http://127.0.0.1:5000 "GET /_alive_04532d2c-bbb3-4cd6-8fac-e670ef387899 HTTP/11" 500 298
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:5000
DEBUG:urllib3.connectionpool:http://127.0.0.1:5000 "GET /_alive_04532d2c-bbb3-4cd6-8fac-e670ef387899 HTTP/11" 500 298
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:5000
DEBUG:urllib3.connectionpool:http://127.0.0.1:5000 "GET /_alive_04532d2c-bbb3-4cd6-8fac-e670ef387899 HTTP/11" 500 298
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:5000
DEBUG:urllib3.connectionpool:http://127.0.0.1:5000 "GET /_alive_04532d2c-bbb3-4cd6-8fac-e670ef387899 HTTP/11" 500 298


---------------------------------------------------------------------------
NotFound                                  Traceback (most recent call last)
NotFound: 404 Not Found: The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.

---------------------------------------------------------------------------
NotFound                                  Traceback (most recent call last)
NotFound: 404 Not Found: The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.

---------------------------------------------------------------------------
NotFound                                  Traceback (most recent call last)
NotFound: 404 Not Found: The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.

---------------------------------------------------------------------------
NotFound                   

DEBUG:urllib3.connectionpool:https://sso.simva-beta.e-ucm.es:443 "GET /realms/simva/.well-known/openid-configuration HTTP/11" 200 6484
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:5000
DEBUG:urllib3.connectionpool:http://127.0.0.1:5000 "GET /_alive_04532d2c-bbb3-4cd6-8fac-e670ef387899 HTTP/11" 500 298
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): minio-api.simva-beta.e-ucm.es:443


---------------------------------------------------------------------------
NotFound                                  Traceback (most recent call last)
NotFound: 404 Not Found: The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.

None - triggered : [{'prop_id': '.', 'value': None}] - Files : [] - Folders : [] - n_clicks_parent : 0


DEBUG:urllib3.connectionpool:https://minio-api.simva-beta.e-ucm.es:443 "POST / HTTP/11" 200 1944
DEBUG:charset_normalizer:Encoding detection: utf_8 is most likely the one.
DEBUG:charset_normalizer:Encoding detection: utf_8 is most likely the one.
DEBUG:botocore.hooks:Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
DEBUG:botocore.hooks:Changing event name from before-call.apigateway to before-call.api-gateway
DEBUG:botocore.hooks:Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
DEBUG:botocore.hooks:Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
DEBUG:botocore.hooks:Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
DEBUG:botocore.hooks:Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.S

...

DEBUG:botocore.hooks:Event choose-service-name: calling handler <function handle_service_name_alias at 0x000002686361E3E0>
DEBUG:botocore.hooks:Event creating-client-class.s3: calling handler <function add_generate_presigned_post at 0x000002686356FB00>
DEBUG:botocore.hooks:Event creating-client-class.s3: calling handler <function lazy_call.<locals>._handler at 0x00000268637DAE80>
DEBUG:botocore.hooks:Event creating-client-class.s3: calling handler <function add_generate_presigned_url at 0x000002686356F880>
DEBUG:botocore.endpoint:Setting s3 timeout as (60, 60)
DEBUG:botocore.client:Registering retry handlers for service: s3
DEBUG:botocore.utils:Registering S3 region redirector handler
DEBUG:botocore.utils:Registering S3Express Identity Resolver
DEBUG:botocore.hooks:Event before-parameter-build.s3.ListObjectsV2: calling handler <function set_list_objects_encoding_type_url at 0x00000268636414E0>
DEBUG:botocore.hooks:Event before-parameter-build.s3.ListObjectsV2: calling handler <function

users/memogames/ - triggered : [{'prop_id': '{"index":"newactivity/","type":"folder-button"}.n_clicks', 'value': 1}] - Files : [] - Folders : [0, 0, 0, 0, 0, 0, 1] - n_clicks_parent : 2


DEBUG:urllib3.connectionpool:https://minio-api.simva-beta.e-ucm.es:443 "GET /traces?list-type=2&prefix=users%2Fmemogames%2Fnewactivity%2F&delimiter=%2F&encoding-type=url HTTP/11" 200 574
DEBUG:botocore.parsers:Response headers: {'Server': 'nginx', 'Date': 'Thu, 18 Jul 2024 10:11:26 GMT', 'Content-Type': 'application/xml', 'Content-Length': '574', 'Connection': 'keep-alive', 'Accept-Ranges': 'bytes', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains, max-age=15768000', 'Vary': 'Origin, Accept-Encoding', 'X-Amz-Id-2': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8', 'X-Amz-Request-Id': '17E346EDCAF5F378', 'X-Content-Type-Options': 'nosniff', 'X-Xss-Protection': '1; mode=block'}
DEBUG:botocore.parsers:Response body:
b'<?xml version="1.0" encoding="UTF-8"?>\n<ListBucketResult xmlns="http://s3.amazonaws.com/doc/2006-03-01/"><Name>traces</Name><Prefix>users/memogames/newactivity/</Prefix><KeyCount>1</KeyCount><MaxKeys>1000</MaxKeys><Delimiter>/</Delimiter><

Folder : {'ResponseMetadata': {'RequestId': '17E346EE0FD36FE5', 'HostId': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'nginx', 'date': 'Thu, 18 Jul 2024 10:11:28 GMT', 'content-type': 'application/xml', 'content-length': '574', 'connection': 'keep-alive', 'accept-ranges': 'bytes', 'strict-transport-security': 'max-age=31536000; includeSubDomains, max-age=15768000', 'vary': 'Origin, Accept-Encoding', 'x-amz-id-2': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8', 'x-amz-request-id': '17E346EE0FD36FE5', 'x-content-type-options': 'nosniff', 'x-xss-protection': '1; mode=block'}, 'RetryAttempts': 0}, 'IsTruncated': False, 'Contents': [{'Key': 'users/memogames/newactivity/xapi-sg-sample-data-array.json', 'LastModified': datetime.datetime(2024, 7, 18, 9, 16, 2, 155000, tzinfo=tzutc()), 'ETag': '"c503c1df4ecb430044d2b846128e1107"', 'Size': 46370, 'StorageClass': 'STANDARD'}], 'Name': 'traces', 'Prefix': 

DEBUG:botocore.hooks:Event choose-service-name: calling handler <function handle_service_name_alias at 0x000002686361E3E0>
DEBUG:botocore.hooks:Event creating-client-class.s3: calling handler <function add_generate_presigned_post at 0x000002686356FB00>
DEBUG:botocore.hooks:Event creating-client-class.s3: calling handler <function lazy_call.<locals>._handler at 0x00000268637DAE80>
DEBUG:botocore.hooks:Event creating-client-class.s3: calling handler <function add_generate_presigned_url at 0x000002686356F880>
DEBUG:botocore.endpoint:Setting s3 timeout as (60, 60)
DEBUG:botocore.client:Registering retry handlers for service: s3
DEBUG:botocore.utils:Registering S3 region redirector handler
DEBUG:botocore.utils:Registering S3Express Identity Resolver
DEBUG:botocore.hooks:Event before-parameter-build.s3.GetObject: calling handler <function sse_md5 at 0x00000268636400E0>
DEBUG:botocore.hooks:Event before-parameter-build.s3.GetObject: calling handler <function validate_bucket_name at 0x00000268

In [ ]:
browser.current_path

In [ ]:
print(f"Folder : {browser.dirs}")
print(f"Files : {browser.files}")